## Overview
We will tune and pilot with the 500 most frequent monosyllabic words from TASA.

In [1]:
# p39 (used CPU implementation)

import numpy as np
import json
import pandas as pd
import time
import tensorflow as tf

from tensorflow.keras import backend as K


from src.learner import *
from utilities import *

# data
kidwords = pd.read_csv('data/kidwords/kidwords.csv', header=None)[0].tolist()

top_500 = pd.read_csv('data/top_500.csv')
test_word_indices = np.array([i for i, e in enumerate(kidwords) if e in top_500])

words, X, Y = subset_kidwords(top_500.word.tolist(), kidwords, np.genfromtxt('data/kidwords/orth.csv', delimiter=","), np.genfromtxt('data/kidwords/phon.csv', delimiter=","), remove_null_columns=True)

2024-08-09 14:58:23.486300: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-09 14:58:23.494908: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-09 14:58:23.504724: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-09 14:58:23.507630: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-09 14:58:23.515193: I tensorflow/core/platform/cpu_feature_guar

GPU check

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


I0000 00:00:1723233508.564497   39425 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1723233508.585530   39425 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1723233508.585656   39425 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


## Tune

In [ ]:
seed = 323


from keras.optimizers import Adam
with open('outputs/tune_top_500_v1.csv', 'w') as f:
    f.write("{},{},{},{},{},{},{},{},{},{},{}\n".format(
                                            "hidden_units",
                                            "learning_rate",
                                             "batch_size",
                                             "epochs",
                                             "loss_train",
                                             "accuracy_train",
                                             "mse_train",
                                             "loss_test",
                                             "accuracy_test",
                                             "mse_test",
                                             "time"))
    for learning_rate in [.001, .005, .01, .025, None]: 
        for batch_size in [10, 20, 30, 40, 50]:
            for epochs in [20, 40, 60]:
                for hidden in [8, 12, 16, 20]:
                    
                    print("Configuration currently training:", learning_rate, batch_size, epochs, hidden)

                    if learning_rate is not None:
                        optimizer = Adam(learning_rate=learning_rate)
                    if learning_rate is None:
                        optimzer = None

                    model = learner(X, Y, seed, hidden, optimizer=None)
                    
                    start_time = time.time()

                    model.fit(X, Y, epochs=epochs, batch_size=batch_size, verbose=True)

                    end_time = time.time()
                    runtime = end_time - start_time

                    loss_train, accuracy_train, mse_train = model.evaluate(X, Y, verbose=0) 

                    f.write("{},{},{},{},{},{},{},{}\n".format(
                                                    hidden,
                                                    learning_rate,
                                                    batch_size,
                                                    epochs,
                                                    loss_train,
                                                    accuracy_train,
                                                    mse_train,
                                                    runtime))
f.close()